# Market Basket Analysis

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
#import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.frequent_patterns import fpgrowth, apriori, association_rules
import networkx as nx

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
dataset_path = '/content/drive/MyDrive/Customer_Segmentation_Project/Data/Gold Layer/uk_data.csv'
uk_data = pd.read_csv(dataset_path, encoding='ISO-8859-1')
uk_data.head(20)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,Date,Time,Year,Month,WeekDay,Quarter,Hour,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2.55,17850.0,United Kingdom,2010-12-01,08:26:00,2010,12,2,4,8,15.30
1,536365,71053,WHITE METAL LANTERN,6,3.39,17850.0,United Kingdom,2010-12-01,08:26:00,2010,12,2,4,8,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2.75,17850.0,United Kingdom,2010-12-01,08:26:00,2010,12,2,4,8,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,3.39,17850.0,United Kingdom,2010-12-01,08:26:00,2010,12,2,4,8,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,3.39,17850.0,United Kingdom,2010-12-01,08:26:00,2010,12,2,4,8,20.34
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,7.65,17850.0,United Kingdom,2010-12-01,08:26:00,2010,12,2,4,8,15.30
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,4.25,17850.0,United Kingdom,2010-12-01,08:26:00,2010,12,2,4,8,25.50
7,536366,22633,HAND WARMER UNION JACK,6,1.85,17850.0,United Kingdom,2010-12-01,08:28:00,2010,12,2,4,8,11.10
8,536366,22632,HAND WARMER RED POLKA DOT,6,1.85,17850.0,United Kingdom,2010-12-01,08:28:00,2010,12,2,4,8,11.10
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,1.69,13047.0,United Kingdom,2010-12-01,08:34:00,2010,12,2,4,8,54.08


In [18]:
# Create a basket for each transaction
basket = (uk_data
          .groupby(['InvoiceNo', 'StockCode'])['Quantity']
          .sum()
          .unstack()
          .reset_index()
          .fillna(0)
          .set_index('InvoiceNo'))
basket = basket.applymap(lambda x: 1 if x > 0 else 0)

basket['TotalItemCount'] = basket.sum(axis=1)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-18-634fa663512b>:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket = basket.applymap(lambda x: 1 if x > 0 else 0)


In [19]:
#basket = basket[basket['TotalItemCount'] > 30]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [20]:
basket.shape

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(16649, 3646)

In [21]:
basket.drop('TotalItemCount', axis=1, inplace=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [22]:
frequent_itemsets = apriori(basket, min_support=0.02, use_colnames=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [23]:
frequent_itemsets

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,support,itemsets
0,0.030212,(20685)
1,0.022043,(20712)
2,0.022284,(20713)
3,0.020422,(20718)
4,0.022944,(20719)
...,...,...
239,0.027509,"(23301, 23300)"
240,0.027509,"(82494L, 82482)"
241,0.020842,"(85099C, 85099B)"
242,0.023725,"(85099B, 85099F)"


In [24]:
num_itemsets = len(frequent_itemsets)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.1, num_itemsets=num_itemsets)
strong_rules = rules[(rules['lift'] >= 1) & (rules['confidence'] >= 0.1)]


# Step 5: Interpret the Rules
strong_rules

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(20725),(20726),0.067271,0.040483,0.021863,0.325000,8.028079,1.0,0.019140,1.421507,0.938576,0.254545,0.296521,0.432530
1,(20726),(20725),0.040483,0.067271,0.021863,0.540059,8.028079,1.0,0.019140,2.027933,0.912373,0.254545,0.506887,0.432530
2,(20727),(20725),0.059823,0.067271,0.029071,0.485944,7.223641,1.0,0.025046,1.814449,0.916387,0.296569,0.448868,0.459043
3,(20725),(20727),0.067271,0.059823,0.029071,0.432143,7.223641,1.0,0.025046,1.655657,0.923704,0.296569,0.396010,0.459043
4,(20725),(20728),0.067271,0.052976,0.024806,0.368750,6.960679,1.0,0.021243,1.500236,0.918097,0.259912,0.333438,0.418502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,"(22697, 22698)",(22699),0.024266,0.040723,0.020482,0.844059,20.726763,1.0,0.019494,6.151553,0.975423,0.460189,0.837439,0.673505
82,"(22699, 22697)",(22698),0.028590,0.029611,0.020482,0.716387,24.192941,1.0,0.019635,3.421518,0.986881,0.542994,0.707732,0.704035
83,(22698),"(22699, 22697)",0.029611,0.028590,0.020482,0.691684,24.192941,1.0,0.019635,3.150691,0.987919,0.542994,0.682609,0.704035
84,(22699),"(22697, 22698)",0.040723,0.024266,0.020482,0.502950,20.726763,1.0,0.019494,1.963050,0.992157,0.460189,0.490589,0.673505


In [28]:
# Calculate revenue for each transaction
uk_data['Revenue'] = uk_data['Quantity'] * uk_data['UnitPrice']

# Calculate total revenue for each product
product_revenue = uk_data.groupby('StockCode')['Revenue'].sum().sort_values(ascending=False)

# Get the top 10 revenue-generating products
top_revenue_products = product_revenue.head(10)

# Display top revenue products for reference
print("Top Revenue-Generating Products:")
print(top_revenue_products)


Top Revenue-Generating Products:
StockCode
23843     168469.60
22423     110713.00
85123A     94960.85
23166      80291.44
85099B     77191.33
47566      63050.28
84879      52228.68
22502      46520.45
79321      45483.91
22086      40392.08
Name: Revenue, dtype: float64


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In the below tabe we have tried to find the products that are assoicated and also if either of the product is in the high revenue category (say A). So that we can give the discounts on the other product (Say B) as A and B are corelated. So our high revenuew products are selling. And when when the customer will saw the offer if on Porduct B if he buys product A then he will be more likely to buy product B

In [29]:
# Filter association rules where at least one item is a top revenue generator
top_revenue_associations = strong_rules[
    strong_rules['antecedents'].apply(lambda x: any(item in top_revenue_products.index for item in x)) |
    strong_rules['consequents'].apply(lambda x: any(item in top_revenue_products.index for item in x))
]

# Display high-revenue, highly associated product pairs
print("\nHigh-Revenue and Strongly Associated Product Pairs:")
top_revenue_associations[['antecedents', 'consequents', 'support', 'confidence', 'lift']]



High-Revenue and Strongly Associated Product Pairs:


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,support,confidence,lift
16,(20725),(85099B),0.022644,0.336607,3.872959
17,(85099B),(20725),0.022644,0.260539,3.872959
34,(21733),(85123A),0.025827,0.669782,5.896985
35,(85123A),(21733),0.025827,0.227393,5.896985
36,(85099B),(21931),0.024506,0.281963,6.519995
37,(21931),(85099B),0.024506,0.566667,6.519995
38,(22910),(22086),0.026248,0.645495,11.372321
39,(22086),(22910),0.026248,0.462434,11.372321
46,(22386),(85099B),0.030392,0.623153,7.169917
47,(85099B),(22386),0.030392,0.349689,7.169917


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
